In [25]:
import os
import sys
sys.path.append(os.path.abspath("../"))
from autogen import initiate_chats, GroupChat, GroupChatManager
from funes.agents.agent_types import Role, Persona, AutogenAgentType

from llm_foundation import logger

from dotenv import find_dotenv, load_dotenv



openai_api_key = os.environ["OPENAI_API_KEY"]

config_list = [{
    "organization": "org-Q3NBjIa66lyyJiZUKXYOXzBf",
    "model": "gpt-3.5-turbo",  # model name
    "api_key": openai_api_key  # api key
}]
llm_config = {
    "seed": 14,  # seed for caching and reproducibility
    "config_list": config_list,  # a list of OpenAI API configurations
    "temperature": 0.0,  # temperature for sampling
}

In [26]:
francisco = Persona.from_json_file("Persona/Francisco.json")
print(francisco)

Persona: Francisco
  Role: learner
  Description: A highly motivated human that is eager to learn.
  Agent System Message: 
  Autogen Code Execution Config: {}



In [27]:
francisco.roles

francisco.save_as_json()

francisco.get_roles()

2024-08-31 07:50:52 fperez-gcloud-stupid-sailor-twift root[2901068] INFO JSON object written: Persona/Francisco.json


['learner']

In [28]:
# francisco_learner = francisco.role_to_autogen_agent("learner", AutogenAgentType.UserProxyAgent, "ALWAYS", llm_config=llm_config)


# francisco_learner.generate_reply(
#     messages=[{"content": "Tell me a joke.", "role": "user"}]
# )


In [29]:
maestro = Persona.from_json_file("Persona/Maestro.json")

print(maestro)

planner = maestro.role_to_autogen_agent("planner", AutogenAgentType.ConversableAgent, llm_config=llm_config) 
maestro_teacher = maestro.role_to_autogen_agent("generic_teacher", AutogenAgentType.AssistantAgent, llm_config=llm_config) 

Persona: La_Parras
  Role: planner
  Description: Given a task, determine what information is needed to complete the task. After each step is done by others, check the progress and instruct the remaining steps
  Agent System Message: Given a task, please determine what information is needed to complete the task. Please note that some of the information will all be retrieved using Python code. After each step is done by others, check the progress and instruct the remaining steps. If a step fails, try to workaround.
  Autogen Code Execution Config: {}
  Role: generic_teacher
  Description: A highly motivated teacher that is eager to offer her knowledge by writing high quality responses.
  Agent System Message: You are a highly motivated teacher. You write engaging and structured text answers following the plan of the planner using markdown format to questions you receive on given topics. You must polish your writing based on the feedback you receive and give a refined version. When the p

In [30]:
engineer = Persona.from_json_file("Persona/Engineer.json")

print(engineer)

sw_engineer = engineer.role_to_autogen_agent("sw_engineer", AutogenAgentType.ConversableAgent, llm_config=llm_config, code_execution_config=False)
code_execution_config = {
    "last_n_messages": 3,
    "work_dir": "coding",
    "use_docker": False,
}
code_executor = engineer.role_to_autogen_agent("code_executor", AutogenAgentType.ConversableAgent, llm_config=llm_config, code_execution_config=code_execution_config)

Persona: Johh_Carmack
  Role: sw_engineer
  Description: An resourceful software enginner.
  Agent System Message: Write code generating graphs/images based on the plan provided by the planner that help to complement the text response of the teacher. Save the resulting artifacts (images/graphs) when necessary in a file and insert the image as markdown in the text sent by the writer. The code should be written in Python and should be clear and concise. The code should be written in a way that it can be easily understood by a learner.
  Autogen Code Execution Config: {}
  Role: code_executor
  Description: Execute the code written by the engineer and report the result.
  Agent System Message: 
  Autogen Code Execution Config: {}



In [31]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

critics = Persona.from_json_file("Persona/Critic.json")

print(critics)

science_critic = critics.role_to_autogen_agent("science_critic", AutogenAgentType.AssistantAgent, llm_config=llm_config)
style_editor = critics.role_to_autogen_agent("style_editor", AutogenAgentType.AssistantAgent, llm_config=llm_config)
ethics_reviewer = critics.role_to_autogen_agent("ethics_reviewer", AutogenAgentType.AssistantAgent, llm_config=llm_config)
meta_reviewer = critics.role_to_autogen_agent("meta_reviewer", AutogenAgentType.AssistantAgent, llm_config=llm_config)

review_chats = [
    {
        "recipient": ethics_reviewer, 
        "message": reflection_message, 
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
        "max_turns": 1},
    {
        "recipient": style_editor, 
        "message": reflection_message, 
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
        "max_turns": 1},    
    {
        "recipient": meta_reviewer, 
        "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
        "max_turns": 1},
]


Persona: Carlos_Boyero
  Role: science_critic
  Description: You are reputable critic of scientific papers and technology.
  Agent System Message: You are a scientific papers and technology critic. You review the work of a writer and provide constructive feedback to help improve the quality of the content.
  Autogen Code Execution Config: {}
  Role: style_editor
  Description: You are reputable critic and editor specialized in text style.
  Agent System Message: As an expert critic and editor, you review and constructively give guidelines to make the text you receive more readable and understandable. Suggest to add markdown format rules to the content parts when necessary for clarity. Begin the review by stating your role.
  Autogen Code Execution Config: {}
  Role: ethics_reviewer
  Description: You are a philosopher specialized in ethics.
  Agent System Message: You are an ethics reviewer, known for your ability to ensure that content is ethically sound and free from any potential et

In [32]:
science_critic.register_nested_chats(
    review_chats,
    trigger=maestro_teacher,
)

In [33]:
task = "What is cross-validation and why is it used in machine learning?"

# chats = [
#     {
#         "sender": francisco_learner,
#         "recipient": maestro_teacher,
#         "message": task,
#         "max_turns": 1,
#         "summary_method": "last_msg",
#     }
    
# ]
# chat_results = initiate_chats(chats)

In [34]:
groupchat = GroupChat(
    agents=[planner, maestro_teacher, science_critic, sw_engineer, code_executor],
    messages=[],
    max_round=20,
    
    allowed_or_disallowed_speaker_transitions={
        planner: [maestro_teacher],
        maestro_teacher: [science_critic, sw_engineer, code_executor],
        sw_engineer: [code_executor],
        code_executor: [sw_engineer, maestro_teacher],
        science_critic: [maestro_teacher]
    },
    speaker_transitions_type="allowed",
    
)

In [35]:
manager = GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

In [36]:
gc_response = science_critic.initiate_chat(
    recipient=manager,
    message=task,
    summary_method="last_msg"
)

Carlos_Boyero_science_critic (to chat_manager):

What is cross-validation and why is it used in machine learning?

--------------------------------------------------------------------------------

Next speaker: La_Parras_generic_teacher

La_Parras_generic_teacher (to chat_manager):

# Cross-Validation in Machine Learning

Cross-validation is a technique used in machine learning to assess how well a model will generalize to an independent dataset. It involves partitioning the dataset into subsets, training the model on a portion of the data, and then evaluating it on the remaining data. This process is repeated multiple times with different partitions, and the performance metrics are averaged to provide a more accurate estimate of the model's performance.

## Why is Cross-Validation Used?

1. **Estimate Model Performance**: Cross-validation provides a more reliable estimate of a model's performance compared to using a single train-test split. It helps in detecting issues like overfittin

2024-08-31 07:50:58 fperez-gcloud-stupid-sailor-twift autogen.agentchat.groupchat[2901068] WARNING The agent 'Johh_Carmack_code_executor' has an empty description, and may not work well with GroupChat.



Next speaker: Carlos_Boyero_science_critic

Carlos_Boyero_science_critic (to chat_manager):

Your explanation of cross-validation in machine learning is clear and concise, providing a good overview of the topic. Here are some suggestions to further enhance the quality of your content:

1. **Provide Examples**: Including examples or code snippets demonstrating how cross-validation is implemented in popular machine learning libraries like scikit-learn can help readers better understand the concept.

2. **Explain Types of Cross-Validation**: Mentioning common types of cross-validation techniques such as k-fold cross-validation, stratified k-fold cross-validation, and leave-one-out cross-validation can provide a more comprehensive understanding of the topic.

3. **Discuss Limitations**: It would be beneficial to address any limitations or potential pitfalls of cross-validation, such as computational complexity or potential biases in certain datasets.

4. **Include Visual Aids**: Incorpora

2024-08-31 07:50:58 fperez-gcloud-stupid-sailor-twift autogen.agentchat.groupchat[2901068] WARNING The agent 'Johh_Carmack_code_executor' has an empty description, and may not work well with GroupChat.



Next speaker: Johh_Carmack_sw_engineer


>>>>>>>> USING AUTO REPLY...
Johh_Carmack_sw_engineer (to chat_manager):

To complement the text provided by the teacher, I will generate a diagram illustrating the K-Fold Cross-Validation technique. This visual aid will help readers better understand how the dataset is divided into subsets for training and testing in this specific cross-validation method.

Here is the Python code to generate the K-Fold Cross-Validation diagram:

```python
import matplotlib.pyplot as plt

# Create a diagram illustrating K-Fold Cross-Validation
plt.figure(figsize=(10, 6))
plt.title("K-Fold Cross-Validation")
plt.xlabel("Data Samples")
plt.ylabel("Model Performance")

# Draw the data samples
plt.scatter(range(100), [1]*100, marker='|', color='blue', label='Data Samples')

# Draw the K-Fold partitions
for i in range(5):
    plt.axvline(x=20*i, color='gray', linestyle='--', linewidth=1)

plt.text(8, 1.1, 'Fold 1', color='gray')
plt.text(28, 1.1, 'Fold 2', color='gr

2024-08-31 07:50:59 fperez-gcloud-stupid-sailor-twift httpx[2901068] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-08-31 07:50:59 fperez-gcloud-stupid-sailor-twift openai._base_client[2901068] INFO Retrying request to /chat/completions in 0.786759 seconds
2024-08-31 07:51:00 fperez-gcloud-stupid-sailor-twift httpx[2901068] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-08-31 07:51:00 fperez-gcloud-stupid-sailor-twift openai._base_client[2901068] INFO Retrying request to /chat/completions in 1.827365 seconds
2024-08-31 07:51:02 fperez-gcloud-stupid-sailor-twift httpx[2901068] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [13]:
gc_response.cost

NameError: name 'gc_response' is not defined

In [ ]:
gc_response.summary

In [ ]:
res = science_critic.initiate_chat(
    recipient=maestro_teacher,
    message=task,
    max_round=10,
    summary_method="last_msg"
)

In [ ]:
print(res.summary)

In [ ]:
for chat_result in res:
    print(chat_result.summary)
    print("\n")

In [ ]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")